# Exemplo

## Modelo cinético de reação de isomerização

Exemplo retirado de Schwaab e Pinto (2007, p.325) que trata sobre a estimação de parâmetros do modelo cinético:

$y = \exp\left[-\left(k_o\cdot 10^{17}\right)\cdot t \cdot \exp\left(-\frac{E}{T}\right)\right]$,                  (1)

em que $y$ é a fração que resta do reagente, $t$ é o tempo, $T$ é a temperatura, por fim $k_o$ e $E$ são os parâmetros a serem estimados.

### Dados experimentais:

Os dados experimentais da variável dependente ($y$) e das variáveis independentes ($t$ e $T$) são disponibilizados em Schwaab e Pinto (2007, p. 325), e apresentados abaixo na forma de listas:

In [93]:
# Tempo
tempo = [120.0,60.0,60.0,120.0,120.0,60.0,60.0,30.0,15.0,60.0,
45.1,90.0,150.0,60.0,60.0,60.0,30.0,90.0,150.0,90.4,120.0,
60.0,60.0,60.0,60.0,60.0,60.0,30.0,45.1,30.0,30.0,45.0,15.0,30.0,90.0,25.0,
60.1,60.0,30.0,30.0,60.0]

# Temperatura
temperatura = [600.0,600.0,612.0,612.0,612.0,612.0,620.0,620.0,620.0,
620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,
620.0,620.0,620.0,620.0,620.0,620.0,631.0,631.0,631.0,631.0,631.0,639.0,639.0,
639.0,639.0,639.0,639.0,639.0,639.0,639.0]

# fração
y = [0.9,0.949,0.886,0.785,0.791,0.890,0.787,0.877,0.938,
0.782,0.827,0.696,0.582,0.795,0.800,0.790,0.883,0.712,0.576,0.715,0.673,
0.802,0.802,0.804,0.794,0.804,0.799,0.764,0.688,0.717,0.802,0.695,0.808,
0.655,0.309,0.689,0.437,0.425,0.638,.659,0.449]

A plataforma MT-PEU necessita da incerteza dos dados experimentais, como essa informação não foi disponibilizada, será assumido o valor 1 para todos os dados:

In [94]:
# lista com incertezas para o tempo
utempo = [1]*41
# lista com incertezas para a temperatura
utemperatura = [1]*41
# lista com incertezas para y
uy = [1]*41

### Modelo:

O modelo é definido na forma de uma subrotina (def) do python e representa a equação (1):

In [95]:
def modelo(param,x,*args):

    tempo = x[:,0:1] # primeira coluna do vetor de dados
    T     = x[:,1:2] # segunda coluna do vetor de dados

    ko = param[0] # primeiro parâmetro
    E  = param[1] # segundo parâmetro

    y = exp(-ko*10**17*tempo*exp(-E/T)) # cálculo de y - vetorizado

    return y

### Configuração da estimação
Importação de pacotes:

In [96]:
# Importando as funções
from MT_PEU import EstimacaoNaoLinear
# Importantando exp
from numpy import exp

Iniciar a função para realizar a estimação:

In [97]:
# Criar o objeto que realiza a estimação
Estime = EstimacaoNaoLinear(modelo, ['y'], ['tempo','Temperatura'], ['ko','E'], PA=0.95, projeto='Projeto')

### Inclusão de dados experimentais na estimação
Incluir os dados experimentais nesse objeto:

In [98]:
# grandezas dependentes. [Opção 0 é para grandeza dependente]
Estime.setDados(0,(tempo,utempo),(temperatura,utemperatura))
# grandezas independentes [Opção 1 é para grandeza independente]
Estime.setDados(1,(y,uy))

Definir que os dados experimentais previamente inseridos serão utilizados como um conjunto de dados para o qual os parâmetros serão estimados:

In [99]:
Estime.setConjunto(tipo='estimacao')


### Executar a otimização (Cálculo dos parâmetros)

Otimização será realizada utilizado o algoritmo Nelder-Mead (simplex) com estimativa inicial de [1, 30000], para $k_o$ e $T$ respectivamente.

In [100]:
Estime.otimiza(estimativa_inicial= [1, 30000],algoritmo='Nelder-Mead')

Para ver o resultado (valor dos parâmetros no ponto ótimo):

In [101]:
# Verificando o resultado da otimização
Estime.parametros.estimativa

[0.8623249732455059, 27642.66282728434]

In [102]:
Estime.Otimizacao.message

'Optimization terminated successfully.'

In [103]:
Estime.Otimizacao.fun

0.010279259412040541

In [104]:
# Avaliando a incerteza
Estime.incertezaParametros(delta=1e-5,metodoIncerteza='SensibilidadeModelo',preencherregiao=False)
Estime.parametros.matriz_covariancia

array([[1.51921418e+03, 1.10691225e+06],
       [1.10691225e+06, 8.06698211e+08]])

## Referência

SCHWAAB, M. M.; PINTO, J. C. Análise de Dados Experimentais I: Fundamentos da Estatística e Estimação de Parâmetros. Rio de Janeiro: e-papers, 2007.